<a href="https://colab.research.google.com/github/jmyth742/AircraftTracker/blob/master/LyndseyCleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## put link here between quotes of the google sheets.
my_sheets = 'https://docs.google.com/spreadsheets/d/1lvAAhyYeQ2cg7es6clQL8LNrRW5fAhRzUgQhd9s1uhw/edit#gid=837286673'
## PLEASE PUT THE USERNAME OF THE CREATOR IN HERE
username = ""

from google.colab import auth
auth.authenticate_user()
import re
from datetime import datetime
import gspread
from google.auth import default
import pandas as pd
from google.colab import files
import numpy as np
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import sys




def flatten(l):
    return [item for sublist in l for item in sublist]


def three_names_regex(links, usernames,):
    
    parts = usernames.split(' ') 
    cleaned_list = []
    left_over = []
    for link in links:
        link = "".join(str(link))
        if re.search(f'(.*){parts[0]}(.*){parts[1]}(.*){parts[1]}(.*)', link): 
            cleaned_list.append(link)
        else:
            left_over.append(link)
    return cleaned_list,left_over



def two_names_regex(links, usernames,):
    
    parts = usernames.split(' ') 
    cleaned_list = []
    left_over = []
    for link in links:
        link = "".join(str(link))
        if re.search(f'(.*){parts[0]}(.*){parts[1]}(.*)', link): 
            cleaned_list.append(link)
        else:
            left_over.append(link)

    return cleaned_list,left_over


def one_names_regex(links, single_name,):
    
    cleaned_list = []
    left_over = []
    for link in links:
        link = "".join(str(link))
        if single_name in link:
            cleaned_list.append(link)
        else:
            left_over.append(link)

    return cleaned_list,left_over


def generate_regex(name,links):
    master_list = []
    left_over_links = []
    
    count = 0
    numbers = name.split(' ')
    count = len(numbers) -1 

    if count == 0 :
        print(f'{count} spaces in {name}')
        master_list,left_over_links = one_names_regex(links,name)
        # for i in range(len(master_list)):
        #     if "https://fapello.com/feed/87616/" in master_list[i]:
        #         print("gotcha")
    elif count == 1:
        print(f'{count} spaces in {name}')
        master_list,left_over_links = two_names_regex(links,name)
    elif count == 2:
        print(f'{count} spaces in {name}')
        master_list,left_over_links = three_names_regex(links,name)

    return master_list,left_over_links


def postProc(creator,my_sheets):

  creds, _ = default()
  gc = gspread.authorize(creds)


  tuple_lists = []
  master_list = []
  left_overs = []

  my_sheets = my_sheets[39:]
  idx = my_sheets.find('/')
  my_sheets = my_sheets[:idx]
  # print(my_sheets)

  clients_sheets = gc.open('Ceartas Customers').sheet1
  filter_names = ''
  removals = ''
  print(creator)
  clients_df = get_as_dataframe(clients_sheets)
  for i in range(len(clients_df['USERNAME'])):
    names_ = clients_df['USERNAME'][i]
    # print(type(names_),i,clients_df['USERNAME'][i])
    names_ = names_.split(',')
    if creator in names_[0]:
      filter_names = clients_df['FILTER_NAMES'][i]
      removals = clients_df['FILTER_REMOVALS'][i] 
    # else:
    #   print("no creator found.")
    #   sys.exit(1)
  worksheet = gc.open('Ceartas Whitelist').sheet1

  sheets = gc.open_by_key(my_sheets)
  # print(sheets)
  links = sheets.sheet1.get_all_values()

  links = flatten(links)

  for j in range(len(links)):
      links[j] = links[j].lower()

  filter_names = filter_names.split(',')  

  for n in filter_names:
      tuple_lists += generate_regex(n,links)



  print(f'{len(tuple_lists)} number of tupes')

  for i in range(0,len(tuple_lists),2):
      master_list += tuple_lists[i]# 0,2,4
      left_overs += tuple_lists[i+1]# 1,3,5
  
  z = list(set(links) - set(master_list))
  remainder = np.array(z, dtype=str)
  np.sort(remainder)
  rem = np.unique(remainder)    
  df_rem = pd.DataFrame(rem)
  rem_links = sheets.add_worksheet(title="remainder",rows=len(rem), cols=1)
  set_with_dataframe(rem_links, df_rem)


  dirty = np.array(master_list, dtype=str)
  np.sort(dirty)
  data = np.unique(dirty)    
  df = pd.DataFrame(data)
  cleaner = sheets.add_worksheet(title="regex_tests",rows=len(data), cols=1)
  set_with_dataframe(cleaner, df) 

  print(f'Stats:\nLength original:{len(links)}\nLength cleaned:{len(data)}, remainder {len(rem)}')


postProc(username,my_sheets)

Lela Sohna
1 spaces in Priyanka Shimar
1 spaces in Lela Sohna
0 spaces in urbabydollxo
8 number of tupes
Stats:
Length original:979
Length cleaned:292, remainder 683
